# orientation check w/o dl

# imports

In [76]:
import os
import sys
import glob
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
sys.path.append("../../")

%run ../../ac/filesystem/greps.py
%run ../../al/feature/face/candide.py

# face module

In [16]:
candide_path = "/home/chy/archive-model/candide/candide.npz"
land_mark_path = "/home/chy/archive-model/candide/shape_predictor_68_face_landmarks.dat"
sample_base_path = "/home/chy/archive-data/processed/frame-angle/train"

In [13]:
candide = Candide(candide_path, land_mark_path)

# collect samples

In [17]:
t_path = "{}/t/".format(sample_base_path)
f_path = "{}/f/".format(sample_base_path)

In [19]:
t_paths = grep_recur(t_path, "*.jpg")
f_paths = grep_recur(f_path, "*.jpg")

# check

In [98]:
def truth_percentage(arr):
    return arr.count(True) / len(arr)

def falsy_percentage(arr):
    return arr.count(False) / len(arr)

def check_portrait(path):
    img = cv2.imread(path)
    return is_portrait(img)

def is_portrait(img):
    fs = candide.get_features(img)
    return fs is not None

def rots_ccw(imgs):
    return [np.rot90(img) for img in imgs]

def guess_orientation(imgs):
    for i, head in enumerate([12, 9, 6, 3]):
        are_portraits = [is_portrait(img) for img in imgs]
        t_percentage = truth_percentage(are_portraits)
        if t_percentage > 0.70:
            return head
        imgs = rots_ccw(imgs)
    return -1

def choose_sample_paths(dir_path, num_samples=20):
    img_paths = glob.glob("{}/*.jpg".format(dir_path))
    return random.sample(img_paths, num_samples)

def read_images(img_paths):
    return [cv2.imread(img) for img in img_paths]

def guess_orientation_of_dir(dir_path):
    img_paths = choose_sample_paths(dir_path)
    sample_imgs = read_images(img_paths)
    return guess_orientation(sample_imgs)

In [83]:
t_portraits = [check_portrait(p) for p in t_paths] 
f_portraits = [check_portrait(p) for p in f_paths] 

In [95]:
print(t_portraits.count(True) / len(t_portraits))
print(f_portraits.count(False) / len(f_portraits))

0.9130434782608695
0.9462809917355371


In [100]:
so = guess_orientation_of_dir(t_path)
print(so)

12
